In [7]:
from agents import Agent, Runner
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
english_agent = Agent(
    name="Assistant", 
    instructions= "You are a helpful assistant. Respond in English",
    model="gpt-4o-mini-2024-07-18"
    )

spanish_agent = Agent(
    name="Assistant", 
    instructions= "Translate it into Latin American Spanish",
    model="gpt-4o-mini-2024-07-18"
    )

In [20]:
english_result = await Runner.run(
    english_agent, 
    "Give me your favorite Bad Bunny song verse"
    )

spanish_result = await Runner.run(
    spanish_agent,
    f"Translate this text to Spanish: {english_result.final_output}"
)

print(f"English Text:\n{english_result.final_output}")
print(f"Spanish Text:\n{spanish_result.final_output}")

English Text:
One of the standout verses by Bad Bunny is from his song "Yo Perreo Sola." In this verse, he emphasizes empowerment and confidence, highlighting the importance of independence and enjoying oneself without needing a partner. The catchy flow and relatable message make it memorable and impactful. If you’d like, I can summarize the themes or discuss its meaning further!
Spanish Text:
Uno de los versos más destacados de Bad Bunny es de su canción "Yo Perreo Sola." En este verso, enfatiza el empoderamiento y la confianza, subrayando la importancia de la independencia y disfrutar uno mismo sin necesidad de una pareja. El ritmo pegajoso y el mensaje identificable lo hacen memorable e impactante. ¡Si quieres, puedo resumir los temas o discutir su significado más a fondo!


Tool Calling definition:

Lets your agents accesse your custom Python-or whatever language you use-functions

In [23]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return f"Weather's good"

weather_agent = Agent(
    name="Weather Agent",
    instructions="You tell the weather",
    tools=[get_weather]
)

weather_result = await Runner.run(
    weather_agent,
    "What's the weather in Medellín?"
)

print(weather_result.final_output)

Getting weather for Medellín
The weather in Medellín is good!


In [ ]:
from agents import WebSearchTool

#Fire Crawl to get free Web Search types

# surf_reporter = Agent(
#     name="Surfer",
#     instructions="You're a surf reporter that finds the most recent data on swell and wind conditions that are instrumental for surfers to know.",
#     tools=[WebSearchTool()]
# )

surf_result = await Runner.run(surf_reporter, "What's the surf report today for Waddell Creek in CA?")
print(surf_result.final_output)

RunResult:
- Last agent: Agent(name="Surfer", ...)
- Final output (str):
    As of Friday, July 18, 2025, at 8:31 PM, the surf conditions at Waddell Creek, California, are as follows:
    
    - **Wave Height**: Approximately 3 to 4 feet. ([deepswell.com](https://deepswell.com/surf-report/US/Santa-Cruz-County/Waddell-Creek/1062?utm_source=openai))
    
    - **Swell**: A combination of long-period swell from the south-southwest (SSW) and short-to-medium period swell from the west-northwest (WNW), with a dominant period of 16 seconds. ([deepswell.com](https://deepswell.com/surf-report/US/Santa-Cruz-County/Waddell-Creek/1062?utm_source=openai))
    
    - **Wind**: Moderate, cross/sideshore from the west at around 6 mph. ([deepswell.com](https://deepswell.com/surf-report/US/Santa-Cruz-County/Waddell-Creek/1062?utm_source=openai))
    
    - **Water Temperature**: Approximately 55°F (13°C). ([deepswell.com](https://deepswell.com/surf-report/US/Santa-Cruz-County/Waddell-Creek/1062?utm_sour

Handoffs:
Automatically pass off result of one agent to another without coding it intentionally

In [40]:
from pydantic import BaseModel

class Translator(BaseModel):
    english: str
    spanish: str

class Lyrics(BaseModel):
    lyrics_english: str

spanish_agent_handoff = Agent(
    name="Assistant", 
    handoff_description="Used for translating languages into Latin American Spanish",
    instructions= (
        "You will be given a field called 'lyrics_english'. Return both the original English lyrics "
        "and a translation in Latin American Spanish. Do not translate the English — keep it as is."
    ),
    model="gpt-4o-mini-2024-07-18",
    output_type=Translator
    )

english_agent_handoff = Agent(
    name="Assistant", 
    instructions= "You are a helpful assistant that knows tons of Latin American and Spanish songs. Return song lyrics in English, then hand it off to the spanish agent.",
    model="gpt-4o-mini-2024-07-18",
    handoffs=[spanish_agent_handoff],
    output_type=Lyrics
    )

handoff_test = await Runner.run(
    english_agent_handoff,
    "Write out a short verse from a Bad Bunny song. Then include a spanish translation of the lyrics below the english lyrics."
)

print(handoff_test.final_output)

english="I don't care that they talk, I just want to feel the rhythm,\nDance until the dawn, lose myself in the beat." spanish='No me importa que hablen, solo quiero sentir el ritmo,\nBailar hasta el amanecer, perderme en el ritmo.'
